In [1]:
import tensorflow.compat.v2 as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pathlib
import random
import glob
import os
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import time
keras = tf.keras
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32)
    image = image/255.0
    image = 2*image-1
    return image

In [3]:
train_dir = r'C:\Users\24306\Desktop\datasets\kfold\3\train'
test_dir  = r'C:\Users\24306\Desktop\datasets\kfold\3\test'
checkpoint_path = r'C:\Users\24306\Desktop\RegNetY008\3\RegNetY008.ckpt'

# 处理训练数据

In [4]:
#打乱顺序
train_root = pathlib.Path(train_dir)
train_image_paths = list(train_root.glob('*/*'))
train_image_paths = [str(path) for path in train_image_paths]
random.shuffle(train_image_paths)
#处理训练标签
train_image_labels = [pathlib.Path(path).parent.name for path in train_image_paths]
train_labels = [[label.split('_')[0]] for label in train_image_labels]
new_train_labels=[]
for n in train_labels:
    new_train_labels.append(float(n[0]));  
train_labels=new_train_labels
print(train_labels)
#处理训练图像
train_path_ds = tf.data.Dataset.from_tensor_slices(train_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_image_ds = train_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
train_label_ds = tf.data.Dataset.from_tensor_slices(train_labels)
train_image_label_ds = tf.data.Dataset.zip((train_image_ds, train_label_ds))
# image_count = len(all_image_paths)

# test_count = int(image_count*0.2)
train_count = len(train_image_paths)
# train_data = image_label_ds.skip(test_count)
# test_data = image_label_ds.take(test_count)

BATCH_SIZE = 32
train_data = train_image_label_ds.shuffle(buffer_size=train_count).repeat(-1)
train_data = train_data.batch(BATCH_SIZE)
train_data = train_data.prefetch(buffer_size=AUTOTUNE)

#test_data = test_data.batch(BATCH_SIZE)
print(train_data)



[37.0, 84.0, 39.0, 37.0, 55.0, 45.0, 80.0, 45.0, 96.0, 13.0, 95.0, 35.0, 10.0, 84.0, 40.0, 84.0, 52.0, 86.0, 45.0, 32.0, 80.0, 16.0, 13.0, 96.0, 30.0, 13.0, 13.0, 30.0, 86.0, 80.0, 40.0, 96.0, 7.0, 100.0, 47.0, 20.0, 80.0, 32.0, 30.0, 84.0, 100.0, 20.0, 96.0, 40.0, 37.0, 37.0, 47.0, 84.0, 13.0, 66.0, 4.0, 16.0, 45.0, 16.0, 86.0, 7.0, 66.0, 7.0, 66.0, 40.0, 4.0, 23.0, 86.0, 66.0, 93.0, 37.0, 84.0, 16.0, 47.0, 39.0, 45.0, 13.0, 20.0, 47.0, 84.0, 45.0, 84.0, 55.0, 52.0, 95.0, 10.0, 10.0, 66.0, 37.0, 96.0, 52.0, 13.0, 32.0, 93.0, 13.0, 66.0, 37.0, 37.0, 4.0, 13.0, 32.0, 66.0, 10.0, 66.0, 45.0, 45.0, 47.0, 47.0, 96.0, 86.0, 37.0, 100.0, 80.0, 84.0, 13.0, 86.0, 40.0, 37.0, 32.0, 80.0, 13.0, 13.0, 10.0, 4.0, 32.0, 52.0, 37.0, 45.0, 96.0, 16.0, 55.0, 55.0, 35.0, 95.0, 93.0, 37.0, 7.0, 23.0, 96.0, 96.0, 10.0, 95.0, 86.0, 47.0, 37.0, 84.0, 47.0, 4.0, 40.0, 35.0, 7.0, 37.0, 95.0, 93.0, 7.0, 47.0, 10.0, 7.0, 23.0, 100.0, 55.0, 7.0, 80.0, 47.0, 16.0, 96.0, 66.0, 37.0, 37.0, 93.0, 86.0, 23.0, 37.0, 

# 处理验证数据

In [5]:
#打乱顺序
test_root = pathlib.Path(test_dir)
test_image_paths = list(test_root.glob('*/*'))
test_image_paths = [str(path) for path in test_image_paths]
#random.shuffle(train_image_paths)
#处理训练标签
test_image_labels = [pathlib.Path(path).parent.name for path in test_image_paths]
test_labels = [[label.split('_')[0]] for label in test_image_labels]
new_test_labels=[]
for n in test_labels:
    new_test_labels.append(float(n[0]));  
test_labels=new_test_labels
print(test_labels)
#处理训练图像
test_path_ds = tf.data.Dataset.from_tensor_slices(test_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
test_image_ds = test_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
test_label_ds = tf.data.Dataset.from_tensor_slices(test_labels)
test_image_label_ds = tf.data.Dataset.zip((test_image_ds, test_label_ds))
# image_count = len(all_image_paths)

# test_count = int(image_count*0.2)
test_count = len(test_image_paths)
# train_data = image_label_ds.skip(test_count)
# test_data = image_label_ds.take(test_count)

BATCH_SIZE = 32
test_data = test_image_label_ds.shuffle(buffer_size=test_count).repeat(-1)
test_data = test_data.batch(BATCH_SIZE)
test_data = test_data.prefetch(buffer_size=AUTOTUNE)

#test_data = test_data.batch(BATCH_SIZE)
print(test_data)

[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.

In [6]:
#要改
RegNetY008 =tf.keras.applications.regnet.RegNetY008(weights='imagenet',include_top=False,input_shape=(224, 224, 3))
#要改
RegNetY008.trainable = True
inputs = tf.keras.Input(shape=(224, 224, 3))
#要改
x = RegNetY008(inputs)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
out_weight = tf.keras.layers.Dense(1,name='out_weight')(x)

model = tf.keras.Model(inputs=inputs,
                       outputs=[out_weight])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 regnety008 (Functional)     (None, 7, 7, 768)         5524056   
                                                                 
 global_average_pooling2d (G  (None, 768)              0         
 lobalAveragePooling2D)                                          
                                                                 
 out_weight (Dense)          (None, 1)                 769       
                                                                 
Total params: 5,524,825
Trainable params: 5,494,937
Non-trainable params: 29,888
_________________________________________________________________


In [7]:
print ('start time:',time.asctime( time.localtime(time.time()) ))
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, decay=0.01),loss='mse')

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,save_best_only=True,monitor='val_loss')
history = model.fit(train_data,
                    steps_per_epoch=train_count//BATCH_SIZE,
                    epochs=300,
                    validation_data=test_data,
                    validation_steps=test_count//BATCH_SIZE,
                    callbacks=[cp_callback])
#要改
np.savetxt('RegNetY008_loss_3.txt',history.history['loss'])
#要改
np.savetxt('RegNetY008_valloss_3.txt',history.history['val_loss'])
print ('end time:',time.asctime( time.localtime(time.time()) ))

start time: Mon Apr 17 17:31:48 2023
Epoch 1/300
591/591 [==============================] - ETA: 0s - loss: 465.8280

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\RegNetY008\3\RegNetY008.ckpt\assets


INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\RegNetY008\3\RegNetY008.ckpt\assets


591/591 [==============================] - 97s 137ms/step - loss: 465.8280 - val_loss: 3.0855
Epoch 2/300
591/591 [==============================] - 64s 108ms/step - loss: 14.9323 - val_loss: 4.2106
Epoch 3/300
591/591 [==============================] - ETA: 0s - loss: 3.8270

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\RegNetY008\3\RegNetY008.ckpt\assets


...

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\RegNetY008\3\RegNetY008.ckpt\assets


591/591 [==============================] - 104s 175ms/step - loss: 0.1481 - val_loss: 0.0706
Epoch 158/300
591/591 [==============================] - 86s 144ms/step - loss: 0.1052 - val_loss: 0.5119
Epoch 159/300
591/591 [==============================] - 85s 144ms/step - loss: 0.1027 - val_loss: 0.1356
Epoch 160/300
591/591 [==============================] - 85s 144ms/step - loss: 0.0727 - val_loss: 0.1190
Epoch 161/300
591/591 [==============================] - 84s 142ms/step - loss: 0.1039 - val_loss: 0.7272
Epoch 162/300
591/591 [==============================] - 84s 143ms/step - loss: 0.0772 - val_loss: 0.4306
Epoch 163/300
591/591 [==============================] - 85s 143ms/step - loss: 0.1295 - val_loss: 0.5361
Epoch 164/300
591/591 [==============================] - 84s 142ms/step - loss: 0.1056 - val_loss: 1.0186
Epoch 165/300
591/591 [==============================] - 85s 143ms/step - loss: 0.1324 - val_loss: 0.2236
Epoch 166/300
591/591 [==============================] - 84

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\RegNetY008\3\RegNetY008.ckpt\assets


INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\RegNetY008\3\RegNetY008.ckpt\assets


591/591 [==============================] - 107s 181ms/step - loss: 0.0730 - val_loss: 0.0620
Epoch 199/300
591/591 [==============================] - 90s 152ms/step - loss: 0.0709 - val_loss: 0.1485
Epoch 200/300
591/591 [==============================] - 90s 153ms/step - loss: 0.0729 - val_loss: 0.1734
Epoch 201/300
591/591 [==============================] - 90s 153ms/step - loss: 0.1234 - val_loss: 0.7865
Epoch 202/300
591/591 [==============================] - 90s 152ms/step - loss: 0.0715 - val_loss: 0.2176
Epoch 203/300
591/591 [==============================] - 91s 154ms/step - loss: 0.0744 - val_loss: 0.1993
Epoch 204/300
591/591 [==============================] - 90s 153ms/step - loss: 0.3018 - val_loss: 0.2235
Epoch 205/300
591/591 [==============================] - 90s 152ms/step - loss: 0.0704 - val_loss: 0.2604
Epoch 206/300
591/591 [==============================] - 90s 153ms/step - loss: 0.1097 - val_loss: 0.1225
Epoch 207/300
591/591 [==============================] - 89